In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

# Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".
Download the data for January and February 2023.
Read the data for January. How many columns are there?

In [2]:
df= pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
nums_columns=len(df.columns)
print(f"{nums_columns} columns")

19 columns


# Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [3]:
df["duration"]= df['tpep_dropoff_datetime']- df['tpep_pickup_datetime']
df['duration'] = df['duration'].dt.total_seconds()/60
print(f'standard deviation: {df["duration"].std():.2f}')

standard deviation: 42.59


# Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [4]:
filtered_df = df[(df['duration'] <= 60) & (df['duration'] >= 1)].copy()
fraction = filtered_df['duration'].count()/df['duration'].count()* 100
print(f"fraction: {fraction:.0f}%")

fraction: 98%


# Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [5]:
filtered_df[["PULocationID", "DOLocationID"]] = filtered_df[["PULocationID", "DOLocationID"]].astype(str)
data=filtered_df[["PULocationID", "DOLocationID"]].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(data) 
print("Number of columns:", X_train.shape[1])

Number of columns: 515


# Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [6]:
y_train = filtered_df['duration'].values
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)
rmse=root_mean_squared_error(y_train, y_pred)
print(f"RMSE on train: {rmse :.2f}")

MemoryError: Unable to allocate 11.5 GiB for an array with shape (3009173, 515) and data type float64

# Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [ ]:
df2= pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
df2["duration"]=df2['tpep_dropoff_datetime']-df2['tpep_pickup_datetime']
df2['duration'] = df2['duration'].dt.total_seconds()/60

filtered_df2 = df2[(df2['duration'] <= 60) & (df2['duration'] >= 1)].copy()
filtered_df2[["PULocationID", "DOLocationID"]] = filtered_df2[["PULocationID", "DOLocationID"]].astype(str)

data2=filtered_df2[["PULocationID", "DOLocationID"]].to_dict(orient='records')

X_train2 = dv.transform(data2) 
y_train2 = filtered_df2['duration'].values
y_pred2 = lr.predict(X_train2)

rmse2= root_mean_squared_error(y_train2, y_pred2)
print(f"RMSE on validation: {rmse2 :.2f}")


RMSE on validation: 7.81
